**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Autoencoder aplicado a una prueba de Estado

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Referencias

1. 

## Contenido

* [Introducción](#Introducción)
* [Importa módulos](#Importa-módulos)
* [Los datos](#Los-datos)
* [Separa áreas](#Separa-áreas)
* [Separa datos de entrenamiento y validación](#Separa-datos-de-entrenamiento-y-validación)
* [Modelo línea base](#Modelo-línea-base)
* [Compila](#Compila)
* [Entrena](#Entrena)
* [Recupera parámetros de los ítems](#Recupera-parámetros-de-los-ítems)
* [Gráficos](#Gráficos)

## Introducción

## Importa módulos

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Input, Dense

from tensorflow.keras.models import Model

from tensorflow.keras.utils import plot_model

from tensorflow.keras.losses import  binary_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.constraints import non_neg

## Los datos

Son 93 items que corresponde a 5 pruebas: 

In [3]:
# Lee los datos
#read csv
datos = pd.read_csv("../Datos/items_93.csv")
areas = pd.read_csv("../Datos/items_areas_93.csv")
# read pickel
#datos = pd.read_pickle("../Datos/items_93.pkl")
#areas = pd.read_pickle("../Datos/items_areas_93.pkl")

In [3]:
datos.head()

,I1,I2,I3,I4,I5,I6,I7,I9,I10,I12,...,I111,I112,I113,I114,I115,I116,I117,I118,I119,I120
0,1,0,0,0,0,0,0,1,0,1,...,1,0,0,1,0,0,1,1,1,0
1,0,0,0,0,0,0,1,1,1,1,...,1,1,1,0,1,1,1,1,0,0
2,0,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,1,1,0
3,1,1,0,1,0,0,1,1,0,1,...,0,0,0,0,1,0,0,0,1,1
4,1,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,1,1,1,1


In [5]:
areas.head()

,item,Id_item,construct_name,construct_nick,Id_construct
0,I1,1,Textual,"""T""",1
1,I2,2,Ciencias,"""C""",3
2,I3,3,Ciencias,"""C""",3
3,I4,4,Ciencias,"""C""",3
4,I5,5,Ciencias,"""C""",3


## Separa áreas

In [6]:
# índices de items
tex_item = areas[areas.Id_construct==1]['item'].values
mat_item = areas[areas.Id_construct==2]['item'].values
cie_item = areas[areas.Id_construct==3]['item'].values
soc_item = areas[areas.Id_construct==4]['item'].values
ima_item = areas[areas.Id_construct==5]['item'].values
# Datos por áreas, no requerido de momento
tex_data = pd.DataFrame(datos[tex_item])
mat_data = pd.DataFrame(datos[mat_item])
cie_data = pd.DataFrame(datos[cie_item])
soc_data = pd.DataFrame(datos[soc_item])
ima_data = pd.DataFrame(datos[ima_item])

## Separa datos de entrenamiento y validación

In [7]:
N = len(datos)# 35562

# Porcentaje de entrenamiento

pe = 0.9           # Porcentaje de Entrenamiento
pt = round(1-pe,2) # Porcentaje de Test

# Número de datos de entrenamiento
train = int(N*pe)
# Número de datos de test
test = N-train

# Fijar semilla para reproducibilidad
np.random.seed(11)
tf.random.set_seed(11)

# Elegir los indices de las filas a seleccionar para entrenamiento de manera aleatoria
train_choice = np.random.choice(N,train,replace=False)

# Extraer datos seleccionados: features = labels
x_train=datos.iloc[train_choice,:]
print("Tamaño de Datos de Entrenamiento:",x_train.shape)

# Elegir los indices de las filas a seleccionar para test 
test_choice=np.setdiff1d(range(N),train_choice)
# Extraer datos seleccionados
x_test=datos.iloc[test_choice,:]


print("Tamaño de Datos de Test         :",x_test.shape)
print("Total de Datos                 :",len(x_train)+len(x_test))

Tamaño de Datos de Entrenamiento: (32005, 93)
Tamaño de Datos de Test         : (3557, 93)
Total de Datos                 : 35562


## Configuración

In [8]:
input_size = x_train.shape[1] #93
hidden_dim = input_size // 2  # 46
latent_dim = 5
num_epochs = 10
batch_size = 64
buffer_size = batch_size * 5

learning_rate = 0.001

## Prepara dataset

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(tf.cast(x_train, dtype = tf.float32))
dataset = dataset.shuffle(buffer_size).batch(batch_size)

## Modelo 

### Clase  VAE

In [10]:
class VAE(tf.keras.Model):

    def __init__(self,dim, dropout=0.2, **kwargs):
        super(VAE, self).__init__(**kwargs)
        
        input_size = dim[0]
        hidden_dim = dim[1]
        latent_dim = dim[2]
        
        self.hidden_layer = tf.keras.layers.Dense(hidden_dim,
                                                  activation='relu',
                                                  name='hidden_layer') # 1
        self.mu_parameter = tf.keras.layers.Dense(latent_dim, name='mu_layer') #2
        self.sigma_parameter = tf.keras.layers.Dense(latent_dim, name='sigma_layer') #3

        #self.fc4 = tf.keras.layers.Dense(hidden_dim) #4   # hidden decoder, no va
        self.output_layer = tf.keras.layers.Dense(input_size, 
                                                  #kernel_constraint=non_neg(),
                                                  name='output_layer',
                                                 activation='sigmoid') #5   # output
        
        self.dropout_1 = tf.keras.layers.Dropout(dropout)
        self.dropout_2 = tf.keras.layers.Dropout(dropout)
        
    def encode(self, x):
        h = self.dropout_1(x)
        h = self.hidden_layer(h)
        h = self.dropout_2(h)  
        
        return self.mu_parameter(h), self.sigma_parameter(h)

    def reparameterize(self, mu, log_var):
        std = tf.exp(log_var * 0.5)
        eps = tf.random.normal(std.shape)

        return mu + eps * std

    #def decode_logits(self, z):
    #    h = self.fc4(z))
    #    return self.fc5(h)

    def decode(self, z):
        #return tf.nn.sigmoid(self.decode_logits(z))
        return self.output_layer(z)

    def call(self, inputs, training=None, mask=None):
        mu, log_var = self.encode(inputs)
        z = self.reparameterize(mu, log_var)
        x_reconstructed_logits = self.decode(z)

        return x_reconstructed_logits, mu, log_var

### Modelo Keras

In [11]:
model = VAE([input_size, hidden_dim, latent_dim])
model.build(input_shape=(3, input_size))
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate)

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer (Dense)         multiple                  4324      
_________________________________________________________________
mu_layer (Dense)             multiple                  235       
_________________________________________________________________
sigma_layer (Dense)          multiple                  235       
_________________________________________________________________
output_layer (Dense)         multiple                  558       
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
Total params: 5,352
Trainable params: 5,352
Non-trainable params: 0
_____________________________________________________________

## Entrenamiento

In [12]:
num_batches = x_train.shape[0] // batch_size

kl_div_list = []
reconstruction_list = []
loss_list = []

for epoch in range(num_epochs):

    for step, x in enumerate(dataset):

        #x = tf.reshape(x, [-1, input_size])

        with tf.GradientTape() as tape:

            # Forward pass
            x_reconstruction_logits, mu, log_var = model(x)
            #print (x_reconstruction_logits, mu, log_var)
            reconstruction_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=x_reconstruction_logits)
            reconstruction_loss = tf.reduce_sum(reconstruction_loss) / batch_size
            kl_div = - 0.5 * tf.reduce_sum(1. + log_var - tf.square(mu) - tf.exp(log_var), axis=-1)
            kl_div = tf.reduce_mean(kl_div)

            # Backprop and optimize
            loss = tf.reduce_mean(reconstruction_loss) + kl_div
            # save losses
            kl_div_list.append(kl_div)
            reconstruction_list.append(reconstruction_loss)
            loss_list.append(loss)

        gradients = tape.gradient(loss, model.trainable_variables)
        for g in gradients:
            tf.clip_by_norm(g, 15)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if (step + 1) % 50 == 0:
            print("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}, Total Loss: {:.4f}"
                  .format(epoch + 1, num_epochs, step + 1, num_batches, float(reconstruction_loss), float(kl_div), float(loss)))

Epoch[1/10], Step [50/500], Reconst Loss: 69.6418, KL Div: 0.3941, Total Loss: 70.0359
Epoch[1/10], Step [100/500], Reconst Loss: 69.1501, KL Div: 0.3187, Total Loss: 69.4688
Epoch[1/10], Step [150/500], Reconst Loss: 68.0315, KL Div: 0.3079, Total Loss: 68.3394
Epoch[1/10], Step [200/500], Reconst Loss: 66.8964, KL Div: 0.5465, Total Loss: 67.4430
Epoch[1/10], Step [250/500], Reconst Loss: 67.3695, KL Div: 0.8511, Total Loss: 68.2206
Epoch[1/10], Step [300/500], Reconst Loss: 65.9233, KL Div: 0.9870, Total Loss: 66.9102
Epoch[1/10], Step [350/500], Reconst Loss: 65.0956, KL Div: 1.0844, Total Loss: 66.1800
Epoch[1/10], Step [400/500], Reconst Loss: 65.2337, KL Div: 1.2330, Total Loss: 66.4667
Epoch[1/10], Step [450/500], Reconst Loss: 64.8367, KL Div: 1.1990, Total Loss: 66.0357
Epoch[1/10], Step [500/500], Reconst Loss: 64.1623, KL Div: 1.2181, Total Loss: 65.3804
Epoch[2/10], Step [50/500], Reconst Loss: 63.4984, KL Div: 1.0924, Total Loss: 64.5908
Epoch[2/10], Step [100/500], Recon

## Recupera parámetros de los ítems

In [ ]:
help(model)

In [14]:
tri_layer= model.get_layer( name='output_layer')
item_params = tri_layer.get_weights()
item_params = np.vstack([np.array(item_params[0]),np.array(item_params[1])])
item_params = pd.DataFrame(np .transpose(item_params))

colnames =['dim_1', 'dim_2','dim_3','dim_4','dim_5','intercepto']

item_params.columns = colnames


In [15]:
item_params.head(50)

,dim_1,dim_2,dim_3,dim_4,dim_5,intercepto
0,0.912029,-0.017002,-0.444610,0.767766,0.627911,0.414429
1,1.207998,-0.015507,-0.565330,0.928492,0.916048,1.205181
2,1.201893,0.603874,-0.372167,-0.649868,0.756885,-2.482927
3,0.933209,0.545310,-0.180608,-0.861281,0.498072,-2.872647
4,0.925277,0.174909,-0.419643,0.176235,0.681475,-0.198225
5,0.763248,0.501723,-0.113317,-0.815546,0.410663,-3.002936
6,1.406333,0.533686,-0.496779,-0.391025,0.964651,-2.226226
7,1.425187,0.693219,-0.455791,-0.718865,0.834543,-2.572235
8,1.294202,0.660563,-0.371222,-0.729657,0.844527,-2.640185
9,0.636204,0.026264,-0.246002,0.336767,0.399698,-0.273801
